# Demo de Deep Autoencoder (DAE) usando capas ConvNet para procesar las imágenes de algunos TIPOS de ANIMALES
Basado en: 

https://blog.keras.io/building-autoencoders-in-keras.html


https://towardsdatascience.com/deep-autoencoders-using-tensorflow-c68f075fd1a3

https://medium.com/analytics-vidhya/building-a-convolutional-autoencoder-using-keras-using-conv2dtranspose-ca403c8d144e

1) Importar librerías:

In [0]:
# nota se debe indicar la versión 1 de TF para compatibilidad del código
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

import keras
from keras.layers import Input, Dense
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Reshape, Flatten
from keras.layers import UpSampling2D, Conv2DTranspose, BatchNormalization
from keras.models import Model
from keras.utils import plot_model

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from PIL import Image

print("\nLibrerías importadas")

2) Definir la configuración del modelo DAE CNN:

In [0]:
# tamaño de las imágenes
# por compatibilidad de tipo de imágenes, para ANIMALES usar sólo en blanco y negro: 1 canal
IMAGE_SHAPE = (128, 128, 1)

# tamaño de los kernels y pooling (para simplificar son todas iguales)
dae_kernel_shape = (3, 3)
dae_pooling_shape = (2, 2)

# indica la configuración para la parte Encoder 
#   (cada elemento de las listas son la configuración de las capas Conv)
dae_filters = [ 32, 64, 64, 64 ]

# la capa de features se define automáticamente 
# en base a la información de la última capa del Encoder
dae_filters.append( 'f' ) 
dae_posLayFeat = len(dae_filters)-1

# indica si el Decoder usa:
#  ver 1: Conv2D + UpSampling (mejores colores pero fuera de foco)
#  ver 2: Conv2DTranspose + BatchNormalization (mejor nitidez pero como manchada)
verCapasDecoder = 2

# cantidad de neuronas ocultas para la parte Decoder 
#   (usa la la lista de Encoder inversa)
for eachEncFilter in dae_filters[0:len(dae_filters)-1][::-1]:
      dae_filters.append( eachEncFilter )

# cantidad de épocas del entrenamiento
cantEpocas = 500

print("Configuración del DAE CNN definida: ")
print ("     -Kernels + Pooling: [", IMAGE_SHAPE, "[", dae_kernel_shape, "+", dae_pooling_shape, "]", IMAGE_SHAPE, "] ")
print ("     -Versión Decoder: ", verCapasDecoder,  " ")
print ("     -Cant. de Filters: [ -,", dae_filters,  ",- ] ")



3) Montar el Drive:

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demo ANIMALES' 
imagPath = path + '/imagenes/train' 

4) Cargar imágenes para entrenar el modelo DAE:

In [0]:
# cargar las imágenes
classes_ori = [] 
images_ori = []

all_dirs = os.listdir( imagPath )
for each_dir in all_dirs:

    auxiPath = imagPath + '/' + each_dir 
    imagFN  = os.listdir( auxiPath )
    for each_imagFN in imagFN:
          
          # abre la imagen
          imag = Image.open(auxiPath + "/" + each_imagFN)
          
          # ajusta el tamaño
          if IMAGE_SHAPE[2]==1:
            imag = imag.convert('L')
            tipoImage = 'L'
          else:
            tipoImage = 'RGB'
          imag = imag.resize((IMAGE_SHAPE[0], IMAGE_SHAPE[1]), Image.ANTIALIAS)          
          
          # transforma a un vector de nros
          arImag = np.array(imag)
          
          # agrega a los vectores
          classes_ori.append( each_dir )
          images_ori.append( arImag )

print("- Clases cargadas: ", len(classes_ori))
print("- Imágenes cargadas: ", len(images_ori))

if len(images_ori)>0:
  print("\n- Ejemplo ", classes_ori[0], " ", images_ori[0].shape, ": ")
  display( Image.fromarray(images_ori[0], tipoImage) )

In [0]:
# define función auxiliar para mostrar imágenes preparadas
def plot_image(imag):
  if IMAGE_SHAPE[2]==1:
    plt.imshow((imag*255).reshape(IMAGE_SHAPE[0], IMAGE_SHAPE[1]).astype(np.uint8))
    plt.gray()
  else:
    plt.imshow((imag*255).reshape(IMAGE_SHAPE).astype(np.uint8))
  plt.axis("off")  

# define función auxiliar para preparar la lista de imágenes a procesar
def prepare_imageList(imagList):    
  auxiAr = np.array(imagList).astype('float32') / 255.
  auxiAr = auxiAr.reshape((len(auxiAr), IMAGE_SHAPE[0], IMAGE_SHAPE[1], IMAGE_SHAPE[2]))  
  return auxiAr

# define vector auxiliar para usar en el entrenamiento
x_train = prepare_imageList(images_ori)

print("x_train (cant ejemplos, datos entrada): ", x_train.shape)
print("\nImagen reconstruida: ")
plot_image(x_train[0])

5) Creación del modelo DAE CNN:

In [0]:
# define la arquitectura de capas del Deep Autoencoder CNN
# teniendo en cuenta la definición dada anteriomente
input_img_Lay = Input(shape=IMAGE_SHAPE, name='input_img') # capa de entrada
eachLay = input_img_Lay
auxName = 'enc_'
auxId = 1 
for i in range(len(dae_filters)):  

    # define el nombre de la capa oculta
    auxlayerName = auxName+str(auxId)
    if i==dae_posLayFeat:
        auxlayerName = 'features'
        auxName = 'dec_'        
    else:        
        if auxName == 'enc_':
          auxId = auxId + 1
        else:
          auxId = auxId - 1

    # agrega las capas ocultas
    if auxlayerName.startswith('enc_'):
          # agrega capa Conv2D + MaxPooling para Encoder
          eachLay = Conv2D(dae_filters[i], dae_kernel_shape, activation='relu', padding='same', name='c_'+auxlayerName)(eachLay) 
          eachLay = MaxPooling2D(dae_pooling_shape, padding='same', name='p_'+auxlayerName)(eachLay)

    elif auxlayerName.startswith('dec_'):
      if verCapasDecoder == 2:
          # Dec v2: agrega capa Conv2DTranspose con BatchNormalization para Decoder        
          eachLay = Conv2DTranspose(dae_filters[i], dae_kernel_shape, strides=2, activation='relu', padding='same', name='c_'+auxlayerName)(eachLay) 
          eachLay = BatchNormalization(name='p_'+auxlayerName)(eachLay)
      else: 
          # Dec v1: agrega capa Conv2 + UpSampling2D para Decoder        
          eachLay = Conv2D(dae_filters[i], dae_kernel_shape, activation='relu', padding='same', name='c_'+auxlayerName)(eachLay) 
          eachLay = UpSampling2D(dae_pooling_shape, name='p_'+auxlayerName)(eachLay)

    elif auxlayerName.startswith('features'):
          #  agrega capa Flatten, Dense y Reshape 
          # para ello utiliza la información del shape de la última capa Encoder
          # y la cantidad de canales de la imagen original
          dae_features_shape = (int(eachLay.shape[1]), int(eachLay.shape[2]), IMAGE_SHAPE[2])
          num_features = dae_features_shape[0]*dae_features_shape[1]*dae_features_shape[2]      

          eachLay = Flatten(name='f_'+auxlayerName)(eachLay)
          eachLay = Dense(num_features, activation='sigmoid', name='d_'+auxlayerName)(eachLay)
          features_Lay = eachLay
          eachLay = Reshape(dae_features_shape, name='r_'+auxlayerName)(eachLay)

# agrega la capa de salida usando la cantidad de canales de la imagen como cantidad de filtros   
output_img_Lay = Conv2D(IMAGE_SHAPE[2], dae_kernel_shape, activation='sigmoid', padding='same', name='output_img')(eachLay)  # capa de salida

# genera el modelo Deep Autoencoder
DAEmodel = Model(input_img_Lay, output_img_Lay, name='DAE CNN')
DAEmodel.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

print("Modelo DAE CNN creado con ", len(DAEmodel.layers), " capas:")
DAEmodel.summary()
print("\n")
plot_model(DAEmodel, show_layer_names=True, show_shapes=True)

5) Entrenar el modelo DAE:

In [0]:
# lleva a cabo el entrenamiento
# usando los mismos datos como entrada y salida
DAEmodel.fit(x_train, x_train,
                epochs = cantEpocas)

7) Evaluar el modelo DAE entrenado solicitando que reconstruya las imágenes ingresadas:

In [0]:
# evalua al modelo 
resEval = DAEmodel.evaluate(x_train, x_train)
print("\n>Evaluación del Modelo: ")
print("    - Error: ", resEval[0])
print("    - Exactitud: ", resEval[1]*100)
print("\n")

# procesa las imágenes con el modelo 
reconstr_imgs = DAEmodel.predict(x_train)

# muestra las 15 primeras imágenes 
print("\n>Resultados: ")
for i in range(len(x_train)):

    # prepara para mostrar
    fig = plt.figure()
    fig.suptitle(classes_ori[i])

    # muestra la real
    ax1 = fig.add_subplot(121)
    plot_image(x_train[i])

    # muestra la generada por el modelo
    ax2 = fig.add_subplot(122)
    plot_image(reconstr_imgs[i])

    plt.tight_layout()
    fig = plt.gcf()


9) Probar el modelo DAE entrenado con otras imágenes:

In [0]:
# función auxiliar para reconstruir la imagen
def reconstruct_image(imag, imagClase):
    
    # prepara y ajusta el tamaño de la imagen
    imagPrep = prepare_imageList([imag])
    
    # ejecuta el modelo
    imagOut = DAEmodel.predict(imagPrep)
    
    # prepara para mostrar
    fig = plt.figure()
    fig.suptitle(imagClase)

    # muestra la real
    ax1 = fig.add_subplot(121)
    plot_image(imagPrep)

    # muestra la generada por el modelo
    ax2 = fig.add_subplot(122)
    plot_image(imagOut)

    plt.tight_layout()
    fig = plt.gcf()


# carga las imágenes de prueba
imagPathPrueba = path + '/imagenes/test' 
all_dirs =  os.listdir( imagPathPrueba ) 
for each_dir in all_dirs:

    auxiPath = imagPathPrueba + '/' + each_dir 
    if os.path.isdir(auxiPath):
      imagFN  = os.listdir( auxiPath )
      for each_imagFN in imagFN:
          
          # abre la imagen
          imag = Image.open(auxiPath + "/" + each_imagFN)
          
          # ajusta el tamaño
          if IMAGE_SHAPE[2]==1:
            imag = imag.convert('L')
          imag = imag.resize((IMAGE_SHAPE[0], IMAGE_SHAPE[1]), Image.ANTIALIAS)          
          
          # transforma a un vector de nros
          arImag = np.array(imag)

          # manda a procesar la imagen cargada
          reconstruct_image(arImag, each_imagFN)
                  

8) A partir del modelo DAE entrenado, generar dos sub-modelos Encoder y Decoder:


*   Generar y usar el modelo Encoder para 'clusterizar' las imágenes de entrenamiento:


In [0]:
## Generar el sub-modelo Encoder para Clustering
## (desde input hasta features)

# reutiliza las capas entrenadas del modelo DAE original
clust_input_Lay = input_img_Lay  # capa de entrada
clust_output_Lay =  features_Lay  # capa de salida

# genera el modelo
CLUSTmodel = Model(input_img_Lay, features_Lay, name='Encoder/Clustering')

print("> Modelo Encoder: ")
CLUSTmodel.summary()
plot_model(CLUSTmodel, show_layer_names=True, show_shapes=True)

In [0]:
# función auxiliar para generar un gráfico 
# con los valores codificados 
# usando PCA para simplificarlos en 2 ejes
def genera_grafico_pca(datos, clases, titulo):
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(datos)
    principalDf = pd.DataFrame(data = principalComponents,
                columns = ['pca_1', 'pca_2'])
    finalDf = pd.concat([principalDf, 
                        pd.DataFrame(clases, columns = ['target'])], 
                        axis = 1)

    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('PCA1', fontsize = 15)
    ax.set_ylabel('PCA2', fontsize = 15)
    ax.set_title(titulo, fontsize = 20)
    for target in set(classes_ori):
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'pca_2'],
                  finalDf.loc[indicesToKeep, 'pca_1'],
                  s = 50)
    ax.legend(set(classes_ori))
    ax.grid()

# procesa las imágenes para recibir el valor codificado de cada una
x_train_encoded = CLUSTmodel.predict(x_train)

# muestra el gráfico con imágenes originales
#genera_grafico_pca(x_train, classes_ori, "Representación de Imágenes Originales")

# codifica los valores codificados como vector
x_train_encoded_reshape = []
for val in x_train_encoded:
  x_train_encoded_reshape.append( val.reshape(num_features) )

# muestra estadísticas de los datos codificados
minArClust = np.empty(num_features)
minArClust.fill(9999.99)
maxArClust = np.empty(num_features)
maxArClust.fill(-9999.99)
sumArClust = np.zeros(num_features)
for val in x_train_encoded_reshape:
  for i in range(num_features):
      sumArClust[i] = sumArClust[i]+val[i]
      if val[i]<minArClust[i]: 
          minArClust[i] = val[i]
      if val[i]>maxArClust[i]: 
          maxArClust[i] = val[i]
print("\n\n> Estadísticas de Clutering de Imágenes codificado en ", num_features," valores: ")
print("- Mínimos:   ", minArClust)
print("- Máximos:   ", maxArClust)
print("- Totales:   ", sumArClust)
print("- Promedios: ", sumArClust/len(x_train_encoded))
print("\n\n")

# muestra el gráfico codificado
genera_grafico_pca(x_train_encoded_reshape, classes_ori, "Representación de Clustering de Imágenes")

*   Generar y usar el modelo Decoder para generar nuevas imágenes similares a las entrenadas:


In [0]:
## Generar el sub-modelo Decoder para Generator
## (desde features hasta output)

# genera una copia del modelo DAE original para evitar romperlo
auxiCloneModel = keras.models.model_from_json(DAEmodel.to_json())
#auxiCloneModel.summary()
input_gen = Input(shape=(num_features,), name='input_gen') # nueva capa de entrada
auxLay_gen = input_gen
for pos in range(len(DAEmodel.layers)):

    # obtiene el nombre de la capa actual
  auxName = DAEmodel.layers[pos].name  
  
  # sólo considera las capas luego de features (decoder y output)
  # para copiar los pesos del DAE original y actualizar la estrcutura
  if auxName.startswith('r_features') or auxName.startswith('c_dec_') or auxName.startswith('p_dec_') or auxName=='output_img':   
    auxiCloneModel.layers[pos].set_weights(DAEmodel.layers[pos].get_weights()) 
    auxLay_gen = auxiCloneModel.layers[pos](auxLay_gen) 

# crea el nuevo modelo Generator
GENmodel = Model(input_gen, auxLay_gen, name = 'Decoder/Generator')

print("> Modelo Decoder: ")
GENmodel.summary()
plot_model(GENmodel, show_layer_names=True, show_shapes=True)

In [0]:
# ejecuta el Generator
#  usando valores definidos al azar como datos de entrada

cantImagenGenerar = 3
consideraEstadClust = True

# genera los datos de entrada
# (como la codificación tiene varias posiciones con ceros 
# se considera que sólo se ponen al azar entre 10% y el 70% de los valores, el resto queda en cero,
# --sino se podría hacer con " np.random.rand(cantImagenGenerar, num_features) "-- )
arX = []
if consideraEstadClust:
    # si están definidas las estadísticas de clustering,
    # genera los valores considerandolas
    print("Genera usando estadísticas de Clustering")
    for i in range(cantImagenGenerar):
        X = np.zeros(num_features)
        for pos in range(num_features):
            if sumArClust[pos]>0:
                X[pos] = np.random.uniform(minArClust[pos],maxArClust[pos])
        arX.append( X )
else:
      # si no están definidas las estadísticas de clustering,
    # genera los valores totalmente al azar
    print("Genera usando valores al azar")
    minRnd = num_features*10//100
    maxRnd = num_features*70//100
    for i in range(cantImagenGenerar):
        X = np.zeros(num_features)
        for j in range(np.random.randint(low=minRnd, high=maxRnd)):
            pos = np.random.randint(low=0, high=num_features-1)
            X[pos] = np.random.normal()
        arX.append( X )


# ejecuta el modelo Generator
imagOut = GENmodel.predict( np.array(arX).reshape((len(arX), num_features)) )  
 
# muestra las imágenes generadas
print("\n> Resultados: ")
for i in range(len(arX)):

    fig = plt.figure()

    # muestra los datos (solo primeros 20)
    ax1 = fig.add_subplot(121)
    datosMostrar = arX[i].reshape(num_features, 1) 
    ax1.table(cellText=datosMostrar[:20], loc='center')   
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)  

    #  muestra reconstrucción
    ax2 = fig.add_subplot(122)
    plot_image(imagOut[i])  

    plt.tight_layout()
    fig = plt.gcf()


*   Combinar los resultados de los modelos Clustering y Generator para mostrar que funcionan juntos como el DAE original:

In [0]:
# prueba el modelo Generator usando como entrada los datos de Clustering
pruebaClust = CLUSTmodel.predict( x_train )
pruebaClust_out = GENmodel.predict(  np.array(pruebaClust).reshape((len(pruebaClust), num_features)) )  
 
# muestra las imágenes generadas
print("\n> Resultados (valores de clustering(primeros 20), imagen recounstrida(grande) y original(chica): ")
for i in range(len(x_train_encoded)):

    fig = plt.figure()

    # muestra los datos
    ax1 = fig.add_subplot(121)
    ax1.table(cellText=pruebaClust[i].reshape(num_features, 1)[:20], loc='center')   
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)  

    #  muestra reconstrucción
    ax2 = fig.add_subplot(122)
    plot_image(pruebaClust_out[i])  

    # muestra imagen original
    ax3 = fig.add_subplot(332)
    plot_image(x_train[i])  

    fig = plt.gcf()
